<p align="center">
    <img src="https://upload.wikimedia.org/wikipedia/commons/7/74/Logo_%C3%89cole_normale_sup%C3%A9rieure_-_PSL_%28ENS-PSL%29.svg"
             alt="ENS-PSL"
             width="400"
             style="margin-right: 30px;"/>
    <img src="https://upload.wikimedia.org/wikipedia/en/3/3f/Qube_Research_%26_Technologies_Logo.svg"
             alt="QRT"
             width="200"/>
</p>


# QRT - Electricity Price Spread Prediction  
**Non-parametric modelling of cross-country electricity price spreads**

## Data Challenge  
**Powered by ENS**

<h3><span style="color:#800000;"><strong>Authored by:</strong> <em>Alexandre Mathias DONNAT, Sr</em></span></h3>

**Currently ranked 352/1075:** without reverse-engineering on *https://challengedata.ens.fr/challenges/97*

The goal of this challenge is to predict a continuous target `TARGET` related to **daily electricity price spreads** between two European countries (France and Germany), based on:

- physical system variables (residual load, renewables, net import/export),
- commodity markets (gas, coal, carbon),
- local weather (wind, solar irradiance, temperature, rain),
- country code and day index.

The platform evaluates submissions with the **Spearman correlation** between our predictions and the true hidden `TARGET` values.  
This metric is rank-based : only the *relative ordering* of predictions matters, not their absolute scale.


## Data description

Three CSV files are provided:

### 1) `x_train.csv` - Physical and market features

Each row represents one country-day, uniquely identified by `ID`.  
Typical columns include (anonymised but consistent):

- `COUNTRY` ∈ {FR, DE}  
- `DAY_ID` (integer time index)  
- residual load: `FR_RESIDUAL_LOAD`, `DE_RESIDUAL_LOAD`  
- renewables: wind, solar, hydro generation  
- net imports/exports  
- fuel prices/returns: `GAS_RET`, `COAL_RET`, `CARBON_RET`  
- weather variables: wind speed, temperature, rain, etc.

This file is used as **input features** for the model.


In [1]:
import pandas as pd

df_x = pd.read_csv("x_train.csv")
df_x.head()

,ID,DAY_ID,COUNTRY,DE_CONSUMPTION,FR_CONSUMPTION,DE_FR_EXCHANGE,FR_DE_EXCHANGE,DE_NET_EXPORT,FR_NET_EXPORT,DE_NET_IMPORT,...,FR_RESIDUAL_LOAD,DE_RAIN,FR_RAIN,DE_WIND,FR_WIND,DE_TEMP,FR_TEMP,GAS_RET,COAL_RET,CARBON_RET
0,1054,206,FR,0.210099,-0.427458,-0.606523,0.606523,NaN,0.692860,NaN,...,-0.444661,-0.172680,-0.556356,-0.790823,-0.283160,-1.069070,-0.063404,0.339041,0.124552,-0.002445
1,2049,501,FR,-0.022399,-1.003452,-0.022063,0.022063,-0.573520,-1.130838,0.573520,...,-1.183194,-1.240300,-0.770457,1.522331,0.828412,0.437419,1.831241,-0.659091,0.047114,-0.490365
2,1924,687,FR,1.395035,1.978665,1.021305,-1.021305,-0.622021,-1.682587,0.622021,...,1.947273,-0.480700,-0.313338,0.431134,0.487608,0.684884,0.114836,0.535974,0.743338,0.204952
3,297,720,DE,-0.983324,-0.849198,-0.839586,0.839586,-0.270870,0.563230,0.270870,...,-0.976974,-1.114838,-0.507570,-0.499409,-0.236249,0.350938,-0.417514,0.911652,-0.296168,1.073948
4,1101,818,FR,0.143807,-0.617038,-0.924990,0.924990,NaN,0.990324,NaN,...,-0.526267,-0.541465,-0.424550,-1.088158,-1.011560,0.614338,0.729495,0.245109,1.526606,2.614378


### 2) `y_train.csv` - Target

For each ID in x_train, this file provides:

- ID – primary key,

- TARGET – scalar value used by the scoring engine.

The exact economic definition of TARGET is not disclosed, but empirically it behaves like a noisy spread-related quantity driven by physical and market fundamentals.

In [2]:
df_y = pd.read_csv("y_train.csv")
df_y.head()

,ID,TARGET
0,1054,0.028313
1,2049,-0.112516
2,1924,-0.180840
3,297,-0.260356
4,1101,-0.071733


### 3) `x_test.csv` - Test set to predict
Same structure as x_train, but without the target.
Our submission must contain for every row:

- ID,
- a single column TARGET with our prediction.

In [3]:
df_test = pd.read_csv("x_test.csv")
df_test.head()

,ID,DAY_ID,COUNTRY,DE_CONSUMPTION,FR_CONSUMPTION,DE_FR_EXCHANGE,FR_DE_EXCHANGE,DE_NET_EXPORT,FR_NET_EXPORT,DE_NET_IMPORT,...,FR_RESIDUAL_LOAD,DE_RAIN,FR_RAIN,DE_WIND,FR_WIND,DE_TEMP,FR_TEMP,GAS_RET,COAL_RET,CARBON_RET
0,1115,241,FR,0.340083,-0.433604,-0.423521,0.423521,0.165333,0.519419,-0.165333,...,-0.222525,-0.513180,-0.182048,-0.982546,-0.876632,0.880491,0.692242,0.569419,-0.029697,-0.929256
1,1202,1214,FR,0.803209,0.780411,0.601610,-0.601610,0.342802,0.555367,-0.342802,...,0.857739,-0.340595,-0.301094,-0.759816,-1.221443,-0.616617,-0.737496,0.251251,0.753646,0.664086
2,1194,1047,FR,0.795540,0.721954,1.179158,-1.179158,1.620928,0.666901,-1.620928,...,0.447967,0.796475,-0.367248,0.376055,-0.483363,0.865138,0.120079,-1.485642,-0.326450,-0.349747
3,1084,1139,FR,0.172555,-0.723427,-0.044539,0.044539,NaN,-0.205276,NaN,...,-0.561295,-0.542606,-0.013291,-0.791119,-0.894309,0.239153,0.457457,-0.746863,2.262654,0.642069
4,1135,842,FR,0.949714,0.420236,0.617391,-0.617391,0.608561,-0.240856,-0.608561,...,0.503567,-0.230291,-0.609203,-0.744986,-1.196282,0.176557,0.312557,-2.219626,-0.509272,-0.488341


## Scoring - Why Spearman correlation?

The leaderboard score is the Spearman correlation coefficient between:

- the true hidden target vector $y$,
- our prediction vector $\hat{y}$.

Formally, Spearman is the Pearson correlation applied to the ranks:

$$\rho_S(y, \hat{y}) = \rho(\text{rank}(y), \text{rank}(\hat{y}))$$

**Key consequences:**

- any strictly monotonic transform of our predictions (e.g. affine change, log, exp) leaves Spearman unchanged,
- what matters is ordering: if we systematically rank "high" target days above "low" ones, the score increases,
- absolute calibration (in the RMSE sense) is irrelevant here.

Our whole pipeline is therefore designed to produce stable, ordered signals rather than perfectly calibrated point forecasts.

## Our modelling strategy

Given the strong noise and limited sample size, we adopt a simple but robust ML pipeline, fully focused on rank structure:

### Target smoothing in time

The raw `TARGET` is extremely noisy day-to-day.

We build a smoother version `TARGET_SMOOTH` via a centered rolling median over 5 days:

$$\tilde{y}_t = \text{median}(y_{t-2}, \ldots, y_{t+2})$$

This acts as a low-pass filter: the model is trained on $\tilde{y}$, but final OOF evaluation is still done against the original `y_raw`.

### Joint preprocessing of train & test

- `x_train` and `x_test` are concatenated to apply consistent transformations,
- `COUNTRY` is encoded into `COUNTRY_CODE` ∈ {0, 1},
- no manual feature engineering here: the idea is to keep the pipeline generic and ML-driven.

### RankGauss transformation on all features

For each numeric column $x$, we apply:

1. **rank transform:** $r_i = \frac{\text{rank}(x_i)}{n+1}$,

2. **projection to a standard normal** via the inverse error function:

$$z_i = \sqrt{2} \, \text{erf}^{-1}(2r_i - 1)$$

This yields approximately Gaussian distributions, strongly robust to outliers and heavy tails.

It also equalises marginal scales across features, which is helpful both for linear models and trees.

### Median imputation

- Missing values are replaced by the train median for each feature,
- The same median is applied to the test set to avoid leakage.

### Two complementary models trained on `TARGET_SMOOTH`

**Ridge regression:**

- Linear model with L2 regularisation,
- Good at capturing large-scale linear dependencies between RankGauss features and the smoothed target.

**LightGBM regressor:**

- Gradient boosting on decision trees,
- Captures non-linearities and interactions (e.g. stress regimes involving residual load, fuel prices, and renewables).

Both are trained in **5-fold KFold cross-validation**, with:

- input: preprocessed features $X$,
- target: smoothed target $y_{\text{smooth}}$,
- out-of-fold predictions: `oof_ridge`, `oof_lgb`.

### Model selection via OOF Spearman

On the OOF predictions, we compute:

$$\rho_{\text{Ridge}} = \text{Spearman}(y_{\text{raw}}, \hat{y}^{\text{OOF}}_{\text{Ridge}})$$

$$\rho_{\text{LGB}} = \text{Spearman}(y_{\text{raw}}, \hat{y}^{\text{OOF}}_{\text{LGB}})$$

Then we define a simple blend:

$$\hat{y}_{\text{ens}} = w \cdot \hat{y}^{\text{OOF}}_{\text{LGB}} + (1 - w) \cdot \hat{y}^{\text{OOF}}_{\text{Ridge}}$$

and search $w \in \{0, 0.05, \ldots, 1\}$ that maximises OOF Spearman:

$$w^* = \arg\max_{w} \text{Spearman}(y_{\text{raw}}, \hat{y}_{\text{ens}}(w))$$

This gives us both:

- a final OOF estimator of leaderboard performance,
- a data-driven blending weight $w^*$ to apply on the test predictions.

### Test-time predictions and submission

For each test sample:

- `pred_ridge` is the KFold mean of Ridge predictions,
- `pred_lgb` is the KFold mean of LightGBM predictions.

We build:

$$\hat{y}_{\text{test}} = w^* \cdot \hat{y}^{\text{test}}_{\text{LGB}} + (1 - w^*) \cdot \hat{y}^{\text{test}}_{\text{Ridge}}$$

and export this as the final `TARGET` in `submission.csv`.


## Limits & Further Ideas

The dataset exhibits a strong level of structural noise, which naturally limits the achievable Spearman correlation.
Even with extensive preprocessing and feature engineering, the relationship between physical drivers and the TARGET remains weakly predictable and highly volatile.

- **Noise-dominated target**
Daily variations often reflect: abrupt regime changes (imports, weather shocks), short-lived dynamics, measurement volatility.
This makes the TARGET only partially explainable from the available features.

- **Diminishing returns of feature engineering**
Additional lags, cross-interactions or rolling windows can provide small local improvements, but past a certain point, new features amplify noise more than they extract signal.
Beyond ~0.20–0.25 Spearman, the model tends to learn fluctuations that are not structurally reproducible.

- **ML models reach a natural ceiling**
Tree-based models, linear models, and blended approaches all converge to a similar performance range, suggesting that the underlying structure is limited. The problem behaves more like a high-variance stochastic process than a smooth regression task.

- **Beyond classical predictive modelling**
Further progress would require moving away from standard supervised learning toward rank-based or search-driven techniques, which focus on optimizing ordering rather than modelling the underlying physics.
Such approaches can increase correlation but are less interpretable and drift away from realistic forecasting objectives.
This notebook focuses on predictive, interpretable, and realistic modelling.
Within this paradigm, performance appears close to the natural ceiling imposed by the data.

# Code - End-to-end pipeline

# 1 Imports & data loading

In [4]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from scipy.stats import rankdata, spearmanr
from scipy.special import erfinv

import lightgbm as lgb

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

pd.set_option("display.max_columns", 200)

# Load data
X_train = pd.read_csv("x_train.csv")
y_train = pd.read_csv("y_train.csv")
X_test  = pd.read_csv("x_test.csv")

print(X_train.shape, y_train.shape, X_test.shape)


(1494, 35) (1494, 2) (654, 35)


# 2. Preprocessing + TARGET_SMOOTH + RankGauss

In [5]:
# 1) Merge to align TARGET and X_train
train = X_train.merge(y_train, on="ID", how="left")
test  = X_test.copy()

# TTemporal sorting
train = train.sort_values("DAY_ID").reset_index(drop=True)

# Targets
y_raw = train["TARGET"].values

# 2) Rolling median centered on 5 days
train["TARGET_SMOOTH"] = (
    train["TARGET"]
        .rolling(window=5, center=True, min_periods=1)
        .median()
)

y_smooth = train["TARGET_SMOOTH"].values

print(train[["DAY_ID", "TARGET", "TARGET_SMOOTH"]].head(10))

# 3) We remove only the raw TARGET from features (we keep TARGET_SMOOTH in train)
train_features = train.drop(columns=["TARGET"])
test_features  = test.copy()

# Concat train + test to apply the same transformations
full = pd.concat([train_features, test_features], axis=0, ignore_index=True)

# 4) Encoding COUNTRY
full["COUNTRY_CODE"] = full["COUNTRY"].map({"FR": 0, "DE": 1}).astype(float)

def rank_gauss(x: np.ndarray) -> np.ndarray:
    """
    Transform x into quasi N(0,1) via rank -> Gaussian quantile.
    """
    r = rankdata(x) / (len(x) + 1.0)
    return np.sqrt(2.0) * erfinv(2.0 * r - 1.0)

# Feature columns (excluding ID and COUNTRY string)
feature_cols = [c for c in full.columns if c not in ["ID", "COUNTRY"]]

print("Number of features used:", len(feature_cols))

# 5) RankGauss on each feature
for col in feature_cols:
    full[col] = rank_gauss(full[col].values)

# 6) Split train / test
n_train = len(train)

full_train = full.iloc[:n_train].reset_index(drop=True)
full_test  = full.iloc[n_train:].reset_index(drop=True)

# 7) Median imputation column by column (computed on TRAIN only)
for col in feature_cols:
    col_train = full_train[col].values

    if np.all(np.isnan(col_train)):
        med = 0.0
    else:
        med = np.nanmedian(col_train)

    full_train[col] = np.where(np.isnan(full_train[col]), med, full_train[col])
    full_test[col]  = np.where(np.isnan(full_test[col]),  med, full_test[col])

# 8) Final check
X    = full_train[feature_cols].values
X_te = full_test[feature_cols].values

print("X shape   :", X.shape)
print("X_te shape:", X_te.shape)
print("Any NaN in X ?   ", np.isnan(X).any())
print("Any NaN in X_te ?", np.isnan(X_te).any())


   DAY_ID    TARGET  TARGET_SMOOTH
0       0  0.108953       0.108953
1       1 -0.063369       0.485112
2       2  0.861270       0.108953
3       2  2.575976       0.861270
4       3  0.068905       1.031308
5       3  7.138604       1.031308
6       5  1.031308       0.068905
7       5  0.026374       0.026374
8       7 -0.118915      -0.021227
9       7 -0.021227      -0.021227
Number of features used: 35
X shape   : (1494, 35)
X_te shape: (654, 35)
Any NaN in X ?    False
Any NaN in X_te ? False


# 3. KFold CV Ridge + LightGBM 

In [6]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

oof_ridge = np.zeros(n_train)
oof_lgb   = np.zeros(n_train)

pred_ridge = np.zeros(len(test))
pred_lgb   = np.zeros(len(test))

best_iters_lgb = []

for fold, (tr_idx, va_idx) in enumerate(kf.split(X, y_smooth), start=1):
    print(f"\n=== Fold {fold} ===")
    
    X_tr, X_va = X[tr_idx], X[va_idx]
    y_tr, y_va = y_smooth[tr_idx], y_smooth[va_idx]
    
    # Ridge
    ridge = Ridge(alpha=1.0)
    ridge.fit(X_tr, y_tr)
    
    oof_ridge[va_idx] = ridge.predict(X_va)
    pred_ridge += ridge.predict(X_te) / kf.n_splits
    
    # LightGBM
    params = {
        "objective": "regression",
        "metric": "rmse",
        "learning_rate": 0.03,
        "num_leaves": 31,
        "max_depth": 4,
        "subsample": 0.8,
        "colsample_bytree": 0.8,
        "min_child_samples": 20,
        "n_estimators": 5000,
        "reg_alpha": 0.3,
        "reg_lambda": 0.2,
        "n_jobs": -1,
        "random_state": 42 + fold,
    }
    
    lgb_model = lgb.LGBMRegressor(**params)
    lgb_model.fit(
        X_tr, y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric="rmse",
        callbacks=[
            lgb.early_stopping(stopping_rounds=200, verbose=False),
        ],
    )
    
    best_iters_lgb.append(lgb_model.best_iteration_)
    
    oof_lgb[va_idx] = lgb_model.predict(X_va, num_iteration=lgb_model.best_iteration_)
    pred_lgb += lgb_model.predict(X_te, num_iteration=lgb_model.best_iteration_) / kf.n_splits
    
    # Spearman fold on TARGET brute (y_raw)
    sp_ridge = spearmanr(oof_ridge[va_idx], y_raw[va_idx]).correlation
    sp_lgb   = spearmanr(oof_lgb[va_idx],   y_raw[va_idx]).correlation
    
    print(f"Spearman Ridge fold : {sp_ridge:.4f}")
    print(f"Spearman LGBM  fold : {sp_lgb:.4f}")
    print(f"Best iter LGBM      : {lgb_model.best_iteration_}")

# ============================
# Summary
# ============================

sp_ridge_oof = spearmanr(oof_ridge, y_raw).correlation
sp_lgb_oof   = spearmanr(oof_lgb,   y_raw).correlation

# Simple Blend
w_grid = np.linspace(0, 1, 21)
best_w = 0.0
best_ens = -2.0

for w in w_grid:
    ens = w * oof_lgb + (1 - w) * oof_ridge
    s = spearmanr(ens, y_raw).correlation
    if s > best_ens:
        best_ens = s
        best_w = w

print("\nSummary OOF Spearman:")
print("Ridge : {:.4f}".format(sp_ridge_oof))
print("LGBM  : {:.4f}".format(sp_lgb_oof))
print("Ens   : {:.4f}".format(best_ens))
print("Best iters LGBM per fold:", best_iters_lgb)
print("Median best_iter LGBM :", int(np.median(best_iters_lgb)))
print("Best blend weight w (LGBM) :", best_w)



=== Fold 1 ===
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000976 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5354
[LightGBM] [Info] Number of data points in the train set: 1195, number of used features: 22
[LightGBM] [Info] Start training from score 0.022219
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

# 4. Final blending & submission

In [7]:
def spearman_corr(a, b):
    return spearmanr(a, b).correlation

# 1) Sanity check : scores OOF with already built predictions 
s_ridge_oof = spearman_corr(y_raw, oof_ridge)
s_lgb_oof   = spearman_corr(y_raw, oof_lgb)

print(f"[CHECK] Spearman OOF Ridge : {s_ridge_oof:.4f}")
print(f"[CHECK] Spearman OOF LGBM  : {s_lgb_oof:.4f}")

# 2) Search for the best blend on the OOF
best_w = 0.0
best_spear = -2.0

for w in np.linspace(0, 1, 21):
    blend_oof = w * oof_lgb + (1 - w) * oof_ridge
    s = spearman_corr(y_raw, blend_oof)
    if s > best_spear:
        best_spear = s
        best_w = w

print(f"[BLEND] Best blend weight w (LGBM) = {best_w:.2f}")
print(f"[BLEND] Spearman OOF blend      = {best_spear:.4f}")

# 3) Blend of TEST predictions
#    (each pred_* has already been averaged over the folds in block 3)
final_pred = best_w * pred_lgb + (1 - best_w) * pred_ridge

print("\nDistribution of final predictions:")
print(pd.Series(final_pred).describe())

# 4) Creation of the submission file
submission = pd.DataFrame({
    "ID": test["ID"].values,
    "TARGET": final_pred
})

submission.to_csv("submission.csv", index=False)
print("\n File 'submission.csv' written in the current folder.")
display(submission.head())


[CHECK] Spearman OOF Ridge : 0.2058
[CHECK] Spearman OOF LGBM  : 0.3573
[BLEND] Best blend weight w (LGBM) = 0.60
[BLEND] Spearman OOF blend      = 0.3721

Distribution of final predictions:
count    654.000000
mean       0.026090
std        0.088282
min       -0.237717
25%       -0.031895
50%        0.020193
75%        0.082717
max        0.316260
dtype: float64

 File 'submission.csv' written in the current folder.


,ID,TARGET
0,1115,0.004327
1,1202,0.092685
2,1194,-0.030089
3,1084,0.003014
4,1135,-0.102029
